In [1]:
#code by Rudi Pendavingh
class TutteGroup:
    def __init__(self,M,char2=False):
        if char2:
            R = [{frozenset():1}]
        else:
            R = [{frozenset():2}]
        for X in M.nonbases():
            if M.rank(X)==M.full_rank()-1: 
                C=set(M.circuit(X))
                D=set(M.cocircuit(M.groundset()-X))
                e = C.pop()
                f = D.pop()
                for g in C:
                    for h in D:
                        R.append(self.cross_ratio(X-set([e,g]),e,g,f,h))
        B = list(M.bases())
        idx = {B[i] : i for i in range(len(B))}
        idx[frozenset()] = len(B)
        T  = matrix(ZZ, len(R)+1,len(B)+1)
        for i in range(len(R)):
            for X, e in R[i].iteritems():
                T[i, idx[X]]=e  
    
        TT = T.hermite_form(include_zero_rows=False)
        
        self._M = M
        self._bases = B
        self._idx = idx
        self._piv = TT.pivots()
        self._tr = TT.nrows()
        self._TT = TT.stack(vector(ZZ,TT.ncols())).sparse_matrix()
        self._free=frozenset([B[i] for i in range(self._TT.ncols()-1) if T.column(i).is_zero()])
        
    def __repr__(self):
        return 'Tutte group of ' + repr(self._M)
      
    def cross_ratio(self,I,a,b,c,d):
        s=0
        if a<c:
            s+=1
        if c<b:
            s+=1
        if b<d:
            s+=1
        if d<a:
            s+=1
        return {I|set([a,c]):1, I|set([b,d]):1, I|set([a,d]):-1, I|set([b,c]):-1, frozenset():s%2}
    
    def is_zero(self, t):
        for B in t.keys():
            if B in self._free:
                return False
        r = self._tr
        for X, e in t.iteritems():
            self._TT[r, self._idx[X]]=e
        i = 0
        jj = 0
        for j in self._piv:
            if self._TT[r, j]:
                if self._TT[r,j]%self._TT[i,j]:
                    self._TT.set_row_to_multiple_of_row(r,r,0)
                    return False
                self._TT.add_multiple_of_row(r, i, -self._TT[r,j]//self._TT[i,j])   
            i += 1
        if not self._TT.row(r).is_zero():
            self._TT.set_row_to_multiple_of_row(r,r,0)
            return False
        return True 
   
    def cross_ratios(self):
        if self._M.full_rank()<2 or self._M.full_corank()<2:
            return
        for F in self._M.flats(self._M.full_rank()-2):
            H = []
            G = set(self._M.groundset()-F)
            while G:
                e = G.pop()
                H.append(e)
                G = G-self._M.closure(F|set([e]))
            
            if len(H)<4:
                continue
            
            I = self._M.max_independent(F)
            for i in range(len(H)):
                a=H[i]
                for j in range(i):
                    b=H[j]
                    for k in range(j):
                        c=H[k]
                        for l in range(k):
                            d=H[l]
                            yield self.cross_ratio(I,a,b,c,d)
                            yield self.cross_ratio(I,a,c,b,d)
                            yield self.cross_ratio(I,a,d,b,c)          
    
    def DW_condition(self):
        for cr in self.cross_ratios():
            if self.is_zero(cr):
                return False
        return True

In [5]:
T = TutteGroup(matroids.named_matroids.TicTacToe())

In [6]:
T.DW_condition()

False

In [12]:
T = TutteGroup(matroids.named_matroids.NonFano(),True)

In [13]:
T.DW_condition()

False

In [10]:
T = TutteGroup(matroids.named_matroids.NonFano(),False)

In [11]:
T.DW_condition()

True